In [ ]:
!pip install stopwordsiso

In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string
import torch.nn as nn
import torch
from torchnlp.datasets import imdb_dataset
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report

In [ ]:
import stopwordsiso as swahilistopwords

In [ ]:
#Load the data
from google.colab import files
files.upload()

Saving Combined Swahili Dataset - train_v0.2.csv to Combined Swahili Dataset - train_v0.2.csv


In [ ]:

df = pd.read_csv('Combined Swahili Dataset - train_v0.2.csv')
#add extra column for fake news
df['fakeNews'] = np.random.choice([0, 1], df.shape[0])
df.head(100)


,content,category,fakeNews
0,"Bodi ya Utalii Tanzania (TTB) imesema, itafan...",uchumi,0
1,"PENDO FUNDISHA-MBEYA RAIS Dk. John Magufuri, ...",kitaifa,1
2,Mwandishi Wetu -Singida BENKI ya NMB imetoa ms...,uchumi,0
3,"TIMU ya taifa ya Tanzania, Serengeti Boys jan...",michezo,1
4,Na AGATHA CHARLES – DAR ES SALAAM ALIYEKUWA K...,kitaifa,1
...,...,...,...
95,Rais Joseph Kabila wa Congo hatogombea tena u...,kimataifa,0
96,"WAZIRI wa Ardhi, Nyumba, Maji, Nishati, Salam...",kitaifa,0
97,BEATRICE KAIZA Staa wa filamu na Bongo Fleva n...,afya,1
98,\n\n \n\tNa JOSEPH SHALUWA\n \n\tMKONGWE kweny...,burudani,0


***View null value***

In [ ]:
df.isnull().sum()

content     1
category    0
fakeNews    0
dtype: int64

In [ ]:
df.drop_duplicates(inplace= True)
df.shape





(29527, 3)

In [ ]:
df.dropna(axis=0,inplace= True)
df.shape

(29526, 3)

In [ ]:
df.shape

(29526, 3)

In [ ]:
#combine important columns
df['combined']=df['content'] + '' + df['category']
df.head()

,content,category,fakeNews,combined
0,"Bodi ya Utalii Tanzania (TTB) imesema, itafan...",uchumi,0,"Bodi ya Utalii Tanzania (TTB) imesema, itafan..."
1,"PENDO FUNDISHA-MBEYA RAIS Dk. John Magufuri, ...",kitaifa,1,"PENDO FUNDISHA-MBEYA RAIS Dk. John Magufuri, ..."
2,Mwandishi Wetu -Singida BENKI ya NMB imetoa ms...,uchumi,0,Mwandishi Wetu -Singida BENKI ya NMB imetoa ms...
3,"TIMU ya taifa ya Tanzania, Serengeti Boys jan...",michezo,1,"TIMU ya taifa ya Tanzania, Serengeti Boys jan..."
4,Na AGATHA CHARLES – DAR ES SALAAM ALIYEKUWA K...,kitaifa,1,Na AGATHA CHARLES – DAR ES SALAAM ALIYEKUWA K...


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
def process_text(text):
  nopunc = [char for char in text if char not in string.punctuation]
  nopunc = ''.join(nopunc)
  #swahilistopwords = stopwords.stopwords("sw")
  stopwords=nltk.corpus.stopwords.words('english')
  
  newStopWords = ['na', 'lakini', 'ingawa', 'ingawaje', 'kwa', 'sababu', 'hadi', 'hata', 'kama', 'ambapo', 'ambamo', 'ambako', 'ambacho', 'ambao', 'ambaye', 'ilhali', 'ya', 'yake', 'yao', 'yangu', 'yetu', 'yenu', 'vya', 'vyao', 'vyake', 'vyangu', 'vyenu', 'vyetu', 'yako', 'hizo', 'mimi', 'sisi', 'wewe', 'nyinyi', 'yeye', 'wao', 'nao', 'nasi', 'nanyi', 'ni', 'alikuwa', 'atakuwa', 'hii', 'hizi', 'zile', 'ile', 'hivi', 'vile', 'za', 'zake', 'zao', 'zenu', 'kwenye', 'katika', 'kwao', 'kwenu', 'kwetu', 'dhidi', 'kati', 'miongoni', 'katikati', 'wakati', 'kabla', 'baada', 'baadaye', 'nje', 'tena', 'mbali', 'halafu', 'hapa', 'pale', 'mara', 'yoyote', 'wowote', 'chochote', 'vyovyote', 'yeyote', 'lolote', 'mwenye', 'mwenyewe', 'lenyewe', 'lenye', 'wote', 'lote', 'vyote', 'nyote', 'kila', 'zaidi', 'hapana', 'ndiyo', 'au', 'ama', 'sio', 'siye', 'tu', 'budi', 'nyingi', 'nyingine', 'wengine', 'mwingine', 'zingine', 'lingine', 'kingine', 'chote', 'sasa', 'basi', 'bila', 'cha', 'chini', 'hapo', 'huku', 'kule', 'humu', 'hivyo', 'hivyohivyo', 'vivyo', 'palepale', 'fauka', 'hiyo', 'hiyohiyo', 'zilezile', 'hao', 'haohao', 'hukuhuku', 'humuhumu', 'huko', 'hukohuko', 'huo', 'huohuo', 'hili', 'hilihili', 'ilikuwa', 'juu', 'karibu', 'kima', 'kisha', 'kutoka', 'kwenda', 'kubwa', 'ndogo', 'kwamba', 'kuwa', 'la', 'lao', 'lo', 'mdogo', 'mkubwa', 'ng’o', 'pia', 'aidha', 'vilevile', 'kadhalika', 'halikadhalika', 'sana', 'pamoja', 'tafadhali', 'wa', 'wake', 'yule', 'wale', 'zangu', 'afanaleki', 'salale', 'oyee', 'yupi', 'ipi', 'lipi', 'ngapi', 'si', 'angali', 'wangali', 'loo', 'ohoo', 'barabara', 'ewaa', 'walahi', 'masalale', 'duu', 'toba', 'mh', 'kumbe', 'ala', 'ebo', 'haraka', 'pole', 'polepole', 'harakaharaka', 'itakuwa', 'mtakuwa', 'tutakuwa', 'labda', 'yumkini', 'haiyumkini', 'yapata', 'takribani', 'hususani', 'yawezekana', 'nani', 'ndani', 'baadhi', 'kuliko', 'mwa', 'hasha', 'moja', 'pili', 'kwanza', 'ili', 'je', 'jinsi', 'ila', 'nini', 'hasa', 'huu', 'zako', 'alisema', 'walikuwa', 'naye', 'watu', 'sauti', 'akasema', 'baadae', 'muda', 'mrefu', 'akatokea', 'akaja', 'wangu', 'kikubwa', 'kiasi', 'mmoja', 'kidogo', 'mpaka']
  stopwords.extend(newStopWords)
  #stopwords.extend(swahilistopwords)

  #returns words that are not in stop word
  clean_words = [word for word in nopunc.split() if word.lower() not in stopwords]

  return clean_words

In [ ]:
df['content'].head(20).apply(process_text)



***FEATURE EXTRACTION***



In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
message_bow = CountVectorizer(analyzer=process_text).fit_transform(df['combined'])

***Split the data for training and testing***

In [ ]:
#splitting the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split (message_bow, df['fakeNews'], test_size = 0.46, random_state=0)

In [ ]:
message_bow.shape

(29526, 382307)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train, y_train)



MultinomialNB()

In [ ]:
print(classifier.predict(X_train))
print(y_train.values)

[0 0 1 ... 0 0 0]
[0 0 1 ... 1 0 0]


***Training the model***

In [ ]:
from sklearn.metrics import classification_report
pred = classifier.predict(X_train)
print(classification_report(y_train,pred))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98      9414
           1       0.98      0.98      0.98      9482

    accuracy                           0.98     18896
   macro avg       0.98      0.98      0.98     18896
weighted avg       0.98      0.98      0.98     18896



***Test the Model***

In [ ]:
from sklearn.metrics import classification_report
pred = classifier.predict(X_test)
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.59      0.60      0.60      6698
           1       0.61      0.60      0.60      6884

    accuracy                           0.60     13582
   macro avg       0.60      0.60      0.60     13582
weighted avg       0.60      0.60      0.60     13582

